In [17]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sb, joblib

In [18]:
# 데이터 불러오기
df = pd.read_csv("data/domestic_customer_data.csv")

In [19]:
# 차량 구매 서비스 플랫폼 -> 회원 정보, 구매 정보 포함
df

,이름,성별,현재 나이,연령대,연락처,거주 지역,구매연도,차량 구매 횟수,고객 평생 가치,브랜드,모델명,기본가격,공장명
0,강주온,남,23,20대 중반,010-5939-5376,인천광역시,2017,1,779930,현대,캐스퍼,23630000,앨라배마공장
1,김경수,남,42,40대 초반,010-9014-3902,광주광역시,2020,5,23907120,현대,아이오닉9,68340000,인도공장
2,김경수,남,42,40대 초반,010-3234-6055,광주광역시,2016,5,23907120,현대,그랜저,69630000,앨라배마공장
3,김경수,남,42,40대 초반,010-7781-8807,광주광역시,2020,5,23907120,현대,투싼,18910000,브라질공장
4,김경수,남,42,40대 초반,010-1577-4165,광주광역시,2016,5,23907120,현대,아이오닉6,52720000,인도네시아공장
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,최민수,여,65,60대 중반,010-3203-2899,전라남도 목포시,2021,5,1863393840,현대,투싼,50610000,중국공장
801,최민수,여,65,60대 중반,010-6721-5920,전라남도 목포시,2017,5,1863393840,현대,베뉴,59880000,울산공장
802,최민수,여,65,60대 중반,010-5179-2418,전라남도 목포시,2025,5,1863393840,현대,쏘나타,46040000,인도네시아공장
803,최민수,여,65,60대 중반,010-4812-2365,전라남도 목포시,2023,5,1863393840,현대,그랜저,63870000,전주공장


In [20]:
df["거주 지역"] = df["거주 지역"].str.split(" ").str[0]

In [21]:
df

,이름,성별,현재 나이,연령대,연락처,거주 지역,구매연도,차량 구매 횟수,고객 평생 가치,브랜드,모델명,기본가격,공장명
0,강주온,남,23,20대 중반,010-5939-5376,인천광역시,2017,1,779930,현대,캐스퍼,23630000,앨라배마공장
1,김경수,남,42,40대 초반,010-9014-3902,광주광역시,2020,5,23907120,현대,아이오닉9,68340000,인도공장
2,김경수,남,42,40대 초반,010-3234-6055,광주광역시,2016,5,23907120,현대,그랜저,69630000,앨라배마공장
3,김경수,남,42,40대 초반,010-7781-8807,광주광역시,2020,5,23907120,현대,투싼,18910000,브라질공장
4,김경수,남,42,40대 초반,010-1577-4165,광주광역시,2016,5,23907120,현대,아이오닉6,52720000,인도네시아공장
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,최민수,여,65,60대 중반,010-3203-2899,전라남도,2021,5,1863393840,현대,투싼,50610000,중국공장
801,최민수,여,65,60대 중반,010-6721-5920,전라남도,2017,5,1863393840,현대,베뉴,59880000,울산공장
802,최민수,여,65,60대 중반,010-5179-2418,전라남도,2025,5,1863393840,현대,쏘나타,46040000,인도네시아공장
803,최민수,여,65,60대 중반,010-4812-2365,전라남도,2023,5,1863393840,현대,그랜저,63870000,전주공장


In [22]:
# 성별, 현재 나이, 거주 지역, 브랜드 선호도, 예산 받아서 예측 -> 모델명

In [23]:
sorted(df["모델명"].unique())

['G70',
 'G80',
 'G90',
 'GV60',
 'GV70',
 'GV80',
 '그랜저',
 '넥쏘',
 '베뉴',
 '스타리아',
 '쏘나타',
 '아반떼',
 '아이오닉5',
 '아이오닉6',
 '아이오닉9',
 '캐스퍼',
 '투싼',
 '펠리세이드']

# 예측 모델 개발

In [24]:
# 전처리 기본 세팅
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [25]:
df

,이름,성별,현재 나이,연령대,연락처,거주 지역,구매연도,차량 구매 횟수,고객 평생 가치,브랜드,모델명,기본가격,공장명
0,강주온,남,23,20대 중반,010-5939-5376,인천광역시,2017,1,779930,현대,캐스퍼,23630000,앨라배마공장
1,김경수,남,42,40대 초반,010-9014-3902,광주광역시,2020,5,23907120,현대,아이오닉9,68340000,인도공장
2,김경수,남,42,40대 초반,010-3234-6055,광주광역시,2016,5,23907120,현대,그랜저,69630000,앨라배마공장
3,김경수,남,42,40대 초반,010-7781-8807,광주광역시,2020,5,23907120,현대,투싼,18910000,브라질공장
4,김경수,남,42,40대 초반,010-1577-4165,광주광역시,2016,5,23907120,현대,아이오닉6,52720000,인도네시아공장
...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,최민수,여,65,60대 중반,010-3203-2899,전라남도,2021,5,1863393840,현대,투싼,50610000,중국공장
801,최민수,여,65,60대 중반,010-6721-5920,전라남도,2017,5,1863393840,현대,베뉴,59880000,울산공장
802,최민수,여,65,60대 중반,010-5179-2418,전라남도,2025,5,1863393840,현대,쏘나타,46040000,인도네시아공장
803,최민수,여,65,60대 중반,010-4812-2365,전라남도,2023,5,1863393840,현대,그랜저,63870000,전주공장


In [26]:
# X, y 분할
# 성별, 현재 나이, 거주 지역, 브랜드 선호도, 예산 받아서 예측 -> 모델명
X = df[["성별", "현재 나이", "거주 지역", "브랜드", "기본가격"]]
y = df["모델명"]

In [27]:
# X = 최근 거래 금액, 거주 지역, 차량 사이즈, 차량 유형, 연료 구분
# y = 최근 구매 제품

In [28]:
X

,성별,현재 나이,거주 지역,브랜드,기본가격
0,남,23,인천광역시,현대,23630000
1,남,42,광주광역시,현대,68340000
2,남,42,광주광역시,현대,69630000
3,남,42,광주광역시,현대,18910000
4,남,42,광주광역시,현대,52720000
...,...,...,...,...,...
800,여,65,전라남도,현대,50610000
801,여,65,전라남도,현대,59880000
802,여,65,전라남도,현대,46040000
803,여,65,전라남도,현대,63870000


In [29]:
y

0        캐스퍼
1      아이오닉9
2        그랜저
3         투싼
4      아이오닉6
       ...  
800       투싼
801       베뉴
802      쏘나타
803      그랜저
804      G70
Name: 모델명, Length: 805, dtype: object

In [30]:
# 인코더 생성
label = LabelEncoder()
onehot = OneHotEncoder()

In [31]:
X = pd.get_dummies(X, columns=["성별", "거주 지역", "브랜드"])

In [32]:
X

,현재 나이,기본가격,성별_남,성별_여,거주 지역_강원도,거주 지역_경기도,거주 지역_경상남도,거주 지역_경상북도,거주 지역_광주광역시,거주 지역_대구,...,거주 지역_서울특별시,거주 지역_울산광역시,거주 지역_인천광역시,거주 지역_전라남도,거주 지역_전라북도,거주 지역_제주특별자치도,거주 지역_충청남도,거주 지역_충청북도,브랜드_제네시스,브랜드_현대
0,23,23630000,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,42,68340000,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,42,69630000,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
3,42,18910000,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4,42,52720000,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,65,50610000,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
801,65,59880000,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
802,65,46040000,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
803,65,63870000,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True


In [33]:
y

0        캐스퍼
1      아이오닉9
2        그랜저
3         투싼
4      아이오닉6
       ...  
800       투싼
801       베뉴
802      쏘나타
803      그랜저
804      G70
Name: 모델명, Length: 805, dtype: object

# 데이터셋 분할

In [34]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# 고객 추천 모델 --> Classification
# GBM, Linear SVM, CatBoost, KNN, GaussianNB, MultinomialNB, BernoulliNB, MLP 필요
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [36]:
lr = LogisticRegression()
svc = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
gnb = GaussianNB()
knn = KNeighborsClassifier()
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
lgb = LGBMClassifier()
cb = CatBoostClassifier()

In [37]:
lr.fit(train_X, train_y)
svc.fit(train_X, train_y)
dtc.fit(train_X, train_y)
rfc.fit(train_X, train_y)
gbc.fit(train_X, train_y)
gnb.fit(train_X, train_y)
knn.fit(train_X, train_y)
lgb.fit(train_X, train_y)
cb.fit(train_X, train_y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000609 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 204
[LightGBM] [Info] Number of data points in the train set: 644, number of used features: 20
[LightGBM] [Info] Start training from score -3.135494
[LightGBM] [Info] Start training from score -2.324564
[LightGBM] [Info] Start training from score -3.033712
[LightGBM] [Info] Start training from score -3.759649
[LightGBM] [Info] Start training from score -3.289645
[LightGBM] [Info] Start training from score -3.759649
[LightGBM] [Info] Start training from score -2.373354
[LightGBM] [Info] Start training from score -5.081404
[LightGBM] [Info] Start training from score -3.902749
[LightGBM] [Info] Start training from score -2.324564
[LightGBM] [Info] Start traini

In [38]:
y_pred_lr = lr.predict(test_X)
y_pred_svc = svc.predict(test_X)
y_pred_dtc = dtc.predict(test_X)
y_pred_rfc = rfc.predict(test_X)
y_pred_gbc = gbc.predict(test_X)
y_pred_gnb = gnb.predict(test_X)
y_pred_knn = knn.predict(test_X)
y_pred_lgb = lgb.predict(test_X)
y_pred_cb = cb.predict(test_X)

In [39]:
# 정확도 확인
print("Logistic Regression 정확도 : ", accuracy_score(test_y, y_pred_lr))
print("SVM 정확도 : ", accuracy_score(test_y, y_pred_svc))
print("Decision Tree 정확도 : ", accuracy_score(test_y, y_pred_dtc))
print("Random Forest 정확도 : ", accuracy_score(test_y, y_pred_rfc))
print("Gradient Boosting 정확도 : ", accuracy_score(test_y, y_pred_gbc))
print("Gaussian Naive Bayes 정확도 : ", accuracy_score(test_y, y_pred_gnb))
print("KNN 정확도 : ", accuracy_score(test_y, y_pred_knn))
print("LightGBM 정확도 : ", accuracy_score(test_y, y_pred_lgb))
print("CatBoost 정확도 : ", accuracy_score(test_y, y_pred_cb))

Logistic Regression 정확도 :  0.12422360248447205
SVM 정확도 :  0.15527950310559005
Decision Tree 정확도 :  0.9006211180124224
Random Forest 정확도 :  0.2360248447204969
Gradient Boosting 정확도 :  0.9751552795031055
Gaussian Naive Bayes 정확도 :  0.17391304347826086
KNN 정확도 :  0.8819875776397516
LightGBM 정확도 :  0.9254658385093167
CatBoost 정확도 :  0.6956521739130435


In [75]:
joblib.dump(dtc, "model/DecisionTree Model.pkl")
joblib.dump(gbc, "model/GradientBoosting Model.pkl")
joblib.dump(lgb, "model/LightGBM Model.pkl")

['model/LightGBM Model.pkl']

### dtc, rfc, gbc, lgb, cb 모델의 정확도가 높음

In [40]:
from sklearn.metrics import confusion_matrix

In [44]:
pd.DataFrame(confusion_matrix(test_y, y_pred_dtc))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,2,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,11,0,0,1,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,20,0,0,0,0,0,0,1,0


In [49]:
pd.DataFrame(confusion_matrix(test_y, y_pred_gbc))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,12,0,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0


In [50]:
pd.DataFrame(confusion_matrix(test_y, y_pred_lgb))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,2,0,1,0,7,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,21,0,0,0,0,0,0,0,0


In [48]:
sorted(df['모델명'].unique())

['G70',
 'G80',
 'G90',
 'GV60',
 'GV70',
 'GV80',
 '그랜저',
 '넥쏘',
 '베뉴',
 '스타리아',
 '쏘나타',
 '아반떼',
 '아이오닉5',
 '아이오닉6',
 '아이오닉9',
 '캐스퍼',
 '투싼',
 '펠리세이드']

In [49]:
test_X

,현재 나이,기본가격,성별_남,성별_여,거주 지역_강원도,거주 지역_경기도,거주 지역_경상남도,거주 지역_경상북도,거주 지역_광주광역시,거주 지역_대구광역시,...,거주 지역_부산광역시,거주 지역_서울특별시,거주 지역_울산광역시,거주 지역_인천광역시,거주 지역_전라남도,거주 지역_전라북도,거주 지역_충청남도,거주 지역_충청북도,브랜드_제네시스,브랜드_현대
192,43,35270000,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
717,69,17000000,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
168,71,20630000,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
522,37,23700000,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
536,24,35410000,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,51,35410000,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
356,29,54930000,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
525,35,19920000,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
748,56,21000000,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [80]:
new_data = np.array([23, 35000000, 1,0, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,1]).reshape(1, 22)

In [81]:
recom_list = []

In [82]:
recom_list.append(dtc.predict(new_data)[0])
recom_list.append(gbc.predict(new_data)[0])
recom_list.append(lgb.predict(new_data)[0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [83]:
recom_list

['투싼', '스타리아', 'G90']

In [84]:
recom_list = list(set(recom_list))

In [85]:
# 추천 결과 하나씩 보여주기
for i in recom_list :
    print(i)

투싼
스타리아
G90


# 비슷한 가격대 전기차 3대 추천

In [52]:
elec_car_compen = {
    "서울특별시": 9000000,
    "부산광역시": 10500000,
    "대구광역시": 11000000,
    "인천광역시": 10600000,
    "광주광역시": 11000000,
    "대전광역시": 12000000,
    "울산광역시": 10500000,
    "경기도 수원시": 10500000,
    "경기도 성남시": 11000000,
    "충청북도 청주시": 14000000,
    "충청남도 천안시": 14000000,
    "전라북도 전주시": 15000000,
    "전라남도 목포시": 15500000,
    "경상북도 포항시": 13000000,
    "경상남도 창원시": 13000000
}

if "연료 구분" in ["전기", "플러그인 하이브리드", "하이브리드"]:
     pass
else: # 추천 시작
     compen = elec_car_compen["서울특별시"]
     # df.loc[df["최근 거래 금액"] <= 45000000 + compen, "최근 구매 제품"].to_numpy()[0].head(3)

In [53]:
compen = elec_car_compen["서울특별시"]
recom_elec = df.loc[(df["최근 거래 금액"] <= 45000000 + compen) & (df["연료 구분"].isin(["전기", "플러그인 하이브리드", "하이브리드"])), "최근 구매 제품"].to_numpy()[0:3]

In [54]:
for i in recom_elec:
    print(i)

IONIQ 6 (CE)
Grandeur (GN7 HEV)
Tucson (NX4 PHEV)


# 프로모션 : 신용카드 혜택 설명하면서 50% 비싼 차 추천 -> 옵션 붙인 걸로 생각하면 됨

In [55]:
prom_data = np.array([35000000 * 1.5, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,0,1,0, 1,0,0]).reshape(1, 24)

In [56]:
prom_list = []

In [57]:
prom_list.append(dtc.predict(prom_data)[0])
prom_list.append(rfc.predict(prom_data)[0])
prom_list.append(gbc.predict(prom_data)[0])
prom_list.append(lgb.predict(prom_data)[0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [58]:
prom_list = list(set(prom_list))

In [59]:
prom_list

['Santa-Fe ™']

In [60]:
# 프로모션 결과와 추천 결과 비교해서, 같은 결과는 제외하고 보여주기
# 모두 다 같으면 애초에 프로모션 안 함
for i in prom_list :
    if i not in recom_list :
        print(i)